In [2]:
from google.colab import drive
import glob

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!ls "/content/drive/My Drive/Deep Learning/Linnaeus 5 256X256/train"

berry  bird  dog  flower  other


In [5]:
root = "/content/drive/My Drive/Deep Learning/Linnaeus 5 256X256"
files = glob.glob(root+'train/*/*.jpg')


In [6]:
import matplotlib.pyplot as plt
from torchvision import transforms
import torch
from torch import nn
import pathlib
from torch.utils.data import DataLoader
from torchvision import *
import torch.optim as optim
import torch.nn.functional as F

In [7]:
data_transform = transforms.Compose([
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])



In [8]:
train_ds = datasets.ImageFolder("/content/drive/My Drive/Deep Learning/Linnaeus 5 256X256/train",
                                           transform=data_transform)
training_generator = torch.utils.data.DataLoader(train_ds,
                                             batch_size=32, shuffle=True)


In [9]:
validation_ds = datasets.ImageFolder("/content/drive/My Drive/Deep Learning/Linnaeus 5 256X256/test",
                                           transform=data_transform)
validation_generator = torch.utils.data.DataLoader(validation_ds,
                                             batch_size=32, shuffle=True)

In [10]:
opt_functional = {'densenet161':lambda x:optim.Adadelta(x.parameters()),
                  'googlenet':lambda x:optim.Adadelta(x.parameters())}
loss_function = nn.CrossEntropyLoss()

In [11]:
class GoogleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.googlenet = models.googlenet(pretrained=True)
        self.fc1 = nn.Linear(1000, 500)
        self.fc2 = nn.Linear(500, 5)
        
    def forward(self, x):
        x = self.googlenet(x)
        x = F.relu(self.fc1(x))
        x = nn.Dropout(p=0.3)(x)   
        x = torch.softmax(self.fc2(x),dim=1)
        return x
    
class Densenet(nn.Module):
    def __init__(self):
        super().__init__()
        self.densenet161 = models.densenet161(pretrained=True)
        self.fc1 = nn.Linear(1000, 500)
        self.fc2 = nn.Linear(500, 5)
        
    def forward(self, x):
        x = self.densenet161(x)
        x = F.relu(self.fc1(x))
        x = nn.Dropout(p=0.3)(x)   
        x = torch.softmax(self.fc2(x),dim=1)
        return x

In [12]:
import numpy as np

In [13]:

max_epochs = 10

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

print(device)
torch.backends.cudnn.benchmark = True
cnn_dic = {'densenet161':Densenet(),
           'googlenet':GoogleNet()}
mdl_pool = {}
for name,cnn in cnn_dic.items():
    print('training ' + name)
    optimizer = opt_functional[name](cnn)
    cnn.to(device)
    train_epoch_loss = [] 
    train_epoch_acc = [] 
    val_epoch_loss = []
    val_epoch_acc = [] 
    for epoch in range(max_epochs):
        cnn.train()
        running_loss = []
        running_acc = []
        for local_batch, local_labels in training_generator:
            local_batch, local_labels = local_batch.to(device), local_labels.to(device)
            optimizer.zero_grad()
            outputs = cnn(local_batch)
            loss = loss_function(outputs, local_labels)
            loss.backward()
            optimizer.step()
            
            running_loss += [loss.item()]
            _, predicted = torch.max(outputs, 1)
            running_acc += [(predicted == local_labels).sum().item()/training_generator.batch_size]
        train_epoch_loss += [np.mean(running_loss)]
        train_epoch_acc += [np.mean(running_acc)]
        
        running_loss = []
        running_acc = []
        with torch.no_grad():
            cnn.eval()
            for local_batch, local_labels in validation_generator:
                local_batch, local_labels = local_batch.to(device), local_labels.to(device)
                outputs = cnn(local_batch)
                loss = loss_function(outputs, local_labels)
                running_loss += [loss.item()]
                _, predicted = torch.max(outputs, 1)
                running_acc += [(predicted == local_labels).sum().item()/validation_generator.batch_size]

        val_epoch_loss += [np.mean(running_loss)]
        val_epoch_acc += [np.mean(running_acc)]
        print('\t[{:d}] loss: {:7.5f} - acc: {:%} - val loss: {:7.5f} - val acc: {:%}'.format(epoch + 1,
                                                                                            train_epoch_loss[-1],
                                                                                            train_epoch_acc[-1],
                                                                                            val_epoch_loss[-1],
                                                                                            val_epoch_acc[-1]))
    
    mdl_pool[name]={}
    mdl_pool[name]['loss'] = train_epoch_loss
    mdl_pool[name]['acc'] = train_epoch_acc
    mdl_pool[name]['val_loss'] = val_epoch_loss
    mdl_pool[name]['val_acc'] = val_epoch_acc

cuda:0
training densenet161
	[1] loss: 1.56604 - acc: 31.881649% - val loss: 1.50122 - val acc: 38.343254%
	[2] loss: 1.56217 - acc: 31.781915% - val loss: 1.55001 - val acc: 30.158730%
	[3] loss: 1.52119 - acc: 35.887633% - val loss: 1.55485 - val acc: 33.482143%
	[4] loss: 1.42779 - acc: 46.193484% - val loss: 1.37246 - val acc: 52.132937%
	[5] loss: 1.38613 - acc: 50.814495% - val loss: 1.39814 - val acc: 49.900794%
	[6] loss: 1.37648 - acc: 51.894947% - val loss: 1.36727 - val acc: 52.876984%
	[7] loss: 1.38046 - acc: 51.612367% - val loss: 1.38438 - val acc: 50.942460%
	[8] loss: 1.36155 - acc: 53.540559% - val loss: 1.30856 - val acc: 58.382937%
	[9] loss: 1.35760 - acc: 53.889628% - val loss: 1.34279 - val acc: 55.753968%
	[10] loss: 1.32663 - acc: 57.247340% - val loss: 1.37660 - val acc: 52.182540%
training googlenet
	[1] loss: 1.43653 - acc: 46.476064% - val loss: 1.56713 - val acc: 32.986111%
	[2] loss: 1.51699 - acc: 38.414229% - val loss: 1.56343 - val acc: 33.779762%
	[3]